# Simple Bayesian Inference Example
### Kevin Kuns (2018)

This notebook gives a basic example of infering the bias of a coin using Bayesian inference.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.lines as mlines
from scipy.stats import beta
from numpy.random import rand
from scipy.integrate import quad

# to check code execution times
from timeit import default_timer as timer

#%matplotlib inline
plt.style.use("../ranaIFO.mplstyle")

## Set up the model

We want to infer the bias $q$ of a coin; that is $q$ is the probability for the coin to come up heads. If $n$ heads come up over $N$ total flips, the likelihood is
$$p(n|q, N)\propto q^n (1 - q)^{N-n}.$$
To get the posterior we also need to pick a prior for $q$:
$$p(q|n, N) \propto q^n (1 - q)^{N-n} p(q).$$
We'll choose three different priors:
* the flat prior: $p(q)\propto 1$. We have no idea what the value of $q$ is prior to the experiment.
* the fair prior: We think the coin's owner is fairly honest have reason to believe that the coin is fair before the experiment. So take $p(q)$ to be peaked around $q=0.5$.
* the biased prior: We know the coin's owner is shady and have reason to believe that the coin is biased, but don't know whether it's biased towards heads or tails. So take $p(q)$ to be peaked around $0$ and $1$ with little support in between.

In [ ]:
q = np.linspace(0, 1, 500)

flatPrior   = lambda q: 1
fairPrior   = lambda q: beta.pdf(q, 30, 30)
biasedPrior = lambda q: beta.pdf(q, 0.1, 0.1)

In [ ]:
def computeNormalizedPosterior(Nhead, Ntot, prior, q):
    post = lambda x: x**Nhead * (1 - x)**(Ntot - Nhead) * prior(x)
    return post(q) / quad(post, 0, 1)[0]

In [ ]:
def plotCoinData(Nhead, Ntot, q0=None):
    fig, ax = plt.subplots(figsize=(11, 8))
    
    flatPosterior   = computeNormalizedPosterior(Nhead, Ntot, flatPrior,   q)
    fairPosterior   = computeNormalizedPosterior(Nhead, Ntot, fairPrior,   q)
    biasedPosterior = computeNormalizedPosterior(Nhead, Ntot, biasedPrior, q)
    
    ax.plot(q, flatPosterior,  c = 'xkcd:Charcoal', label='flat')
    ax.plot(q, fairPosterior,   'C2:',  label='fair')
    ax.plot(q, biasedPosterior, 'C1-.', label='biased')
    if q0:
        ax.axvline(q0, color='k', lw = 4, alpha = 0.3, label='true value')
    handles, labels = ax.get_legend_handles_labels()
    l1 = mlines.Line2D([], [], alpha=0)
    handles.append(l1)
    labels.append(r'$N = {:d};\; n = {:d}$'.format(Ntot, Nhead))
    ax.legend(handles, labels)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 16)
    ax.set_xlabel(r'probability $q$ of getting heads')
    ax.set_title(r'posterior distribution')

In [ ]:
q0 = 0.3  # The true value of the bias
#np.seterr(all='raise')
Nh = np.cumsum(rand(1000) < q0)

## Plot some results

Before any coins have been tossed the posterior is just the prior.

In [ ]:
fig = plotCoinData(0, 0)

Now look at how the posterior is updated after the first few coin tosses.

In [ ]:
# first 5
fig = plotCoinData(Nh[4], 5, q0);

In [ ]:
# first 10
fig = plotCoinData(Nh[9], 10, q0);

In [ ]:
# first 50
fig = plotCoinData(Nh[49], 50, q0);

In [ ]:
# first 100
fig = plotCoinData(Nh[90], 100, q0);

In [ ]:
# first 200
fig = plotCoinData(Nh[199], 200, q0);

In [ ]:
fig = plotCoinData(Nh[999], 1000, q0);